# EMSCA

This notebook can help you capture and compute metrics on the 

In [ ]:
import sys
sys.path.insert(1,'../../SCApeGoat-main/')

from WPI_SCA_LIBRARY.CWScope import *
from WPI_SCA_LIBRARY.LeakageModels import *
%run "../function/CEMA_functions.ipynb""

In [ ]:
#Configure the cryptocore from the above options 
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET='TINYAES128C'
CRYPTO_OPTIONS=''
SS_VER='SS_VER_1_1'

### Setup the devices

generate the hex file for the crypto core and load it to the target device. For supported options, check the readme file. 

In [ ]:
%%capture #uncomment this for checking output if something is not working 
%run "./Setup_script.ipynb" 

#### Initialize scapegoat scope and parent directory 

In [ ]:
#initiazlize the scope and program the target and husky
# scope.disconnect()
scope = CWScope(fw_path, target_type=cw.targets.SimpleSerial,target_programmer=prog)

In [ ]:
emsca = FileParent("CEMA",".\\",True)
pt = emsca.get_experiment("pt_keys")


#debug scripts to test the datasets
# keys_pt = pt.get_dataset("keys").read_data(0,10000)
# random_pt = pt.get_dataset("plaintexts").read_data(0,10000)
# fixed_pt = pt.get_dataset("fixed_pt").read_data(0,10000)

#used to capture keys and plaintexts
# traces, keys, plaintexts, ciphertexts = scope.standard_capture_traces(5000,fixed_pt=True)

# Motor setup


In [ ]:
#initialise the motor
A=XYZ()
X,Y,Z,interface=A.XYZ_setup(velocity=10000,acceleration=10000)

In [ ]:
#some methods to use for going to starting position, find more functions at https://github.com/analogdevicesinc/PyTrinamic
Z.get_actual_position() #gives actual position 
X.move_by(-stepsize*10) #moves by specific steps

In [ ]:
# side length of CW-lite processor in stepper steps
s_len = 160000*12

# number of steps per axis. square of this number is total steps
n_steps = 10

# size of steps between measurement points
stepsize = - s_len // n_steps
print('step size =', stepsize)

# Capture Traces

In [ ]:
#add/ get experiment 

from datetime import datetime

# Generate a timestamp with date, hour, and minute
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

# test = emsca.get_experiment("Testing_CEMA_grid")
# emsca.delete_experiment("Testing_CEMA_grid_32_32-1")
# test = emsca.add_experiment("Testing_CEMA_grid_10_10_1_25mm_10k"+ timestamp)

# test = emsca.add_experiment("Testing_CEMA_grid_10_10_1_25mm_10k")



In [ ]:
#starts from the current location traces grids and return to the starting location after capturing the whole grid 
Grid_Tracing_scapegoat(stepsize,stepsize,n_steps-1,n_steps-1,X,Y,Z,interface,scope,pt,test,10000)

# Metrics Computation

In [ ]:
#computes TVLA and plots in a heatmap
t = plot_t_statistic_heatmap(test, grid_size=n_steps)

In [ ]:
#computes CEMA and plots in a heatmap
CEMA,CEMA_max = plot_CEMA_heatmap(test,pt,10000,grid_size=n_steps)

In [ ]:
#computes SNR and plots in a heatmap
SNR,SNR_db = plot_SNR_heatmap_byte(test,pt,10000,grid_size=n_steps)

In [ ]:
#Computes box_plot for the given trace numbers
num_list = [500,1000,5000, 10000]  # Different trace numbers to test
a =generate_box_plots(test, pt, num_list, target_byte=0, grid_size=n_steps)


In [ ]:
#computes correaltion vs no. of traces for the specified location
X = 3
Y =  4
a =plot_CEMA_wr(test,pt,800,x = Y,y=9-X,div=10)

# Save Matrix

In [ ]:
save_mat(SNR,'SNR_10k_10x10_1_27_25.mat')

# Checking the results and finding the correct keys

In [ ]:
import pandas as pd
k = pt.get_dataset("Keys").read_data(0,1)
print(k[0])

# Find the coordinates where the value matches the key
coordinates = np.where(CEMA == k[0][0])

# Convert the coordinates to a more readable format (row, column)
coordinates = list(zip(coordinates[0], coordinates[1]))

print(coordinates)

# Step 2: Retrieve scores from CEMA_max for the matching locations
scores = [CEMA_max[coord] for coord in coordinates]

# Step 3: Create a table (DataFrame) with the coordinates and corresponding scores
score_table = pd.DataFrame({
    'Row': [coord[0] for coord in coordinates],
    'Column': [coord[1] for coord in coordinates],
    'Score': scores
})

print(score_table)


# Step 1: Flatten the CEMA_max array and get the indices of the top 10 scores
top_indices = np.unravel_index(np.argsort(CEMA_max.ravel())[-20:], CEMA_max.shape)

# Step 2: Retrieve the corresponding keys from CEMA_guesses for those indices
top_keys = [CEMA[coord] for coord in zip(*top_indices)]

# Step 3: Create a table (DataFrame) with the top scores and their corresponding keys
top_scores = CEMA_max[top_indices]  # Extract top 10 scores

score_table = pd.DataFrame({
    'Row': top_indices[0],
    'Column': top_indices[1],
    'Score': top_scores,
    'Key': top_keys
})

score_table_sorted = score_table.sort_values(by='Score', ascending=False)
# Display the resulting table
print(score_table_sorted)

In [ ]:

# Step 1: Flatten the CEMA_max array and get the indices of all the scores sorted by value
flat_indices = np.unravel_index(np.argsort(CEMA_max.ravel())[::-1], CEMA_max.shape)

# Step 2: Retrieve the corresponding keys from CEMA_guesses for those sorted indices
sorted_keys = [CEMA[coord] for coord in zip(*flat_indices)]

# Step 3: Create a table (DataFrame) with the sorted scores and their corresponding keys
sorted_scores = CEMA_max[flat_indices]  # Extract all scores in sorted order

# Create the DataFrame
sorted_table = pd.DataFrame({
    'Row': flat_indices[0],
    'Column': flat_indices[1],
    'Score': sorted_scores,
    'Key': sorted_keys
})

# Step 4: Find the rank of the key you are looking for
rank = sorted_table[sorted_table['Key'] == k[0][0]].index[0] + 1  # +1 to get the 1-based rank

# Display the sorted table and the rank
print("Sorted Table:")
print(sorted_table)
print(f"\nRank of key {k[0][0]}: {rank}")